# El-Gamal Encryption Scheme

## Group-11 
+ Ritu Thombre (BT17CSE084)

# <a id='index'>Index</a>

+ <a href='#primitive_roots'>1. Finding primitive roots of a prime</a>
+ <a href='#key'>2. Generation of Keys</a>
+ <a href='#encrypt'>3. Encrypting plain_text</a>
+ <a href='#decrypt'>4. Decrypting cipher_text</a>
+ <a href='#aux'>5. Auxiliary functions</a>
+ <a href='#implementation'>6. Implementation</a>
    + <a href='#6.1'>6.1 Generating Keys</a>
    + <a href='#6.2'>6.2 Generating plain_text</a>
    + <a href='#6.3'>6.3 Encrypting plain_text to generate cipher_text</a>
    + <a href='#6.4'>6.4 Decrypting cipher_text to generate plain_text</a>
    + <a href='#6.5'>6.5 Checking if decrypted plain_text is same as original plain_text</a>
+ <a href='#final'>7. Final Remarks</a>

In [28]:
import numpy as np
from Crypto.Util import number
import math
import gmpy2
from gmpy2 import powmod,mpz,isqrt,invert
import pyecm

# <a id='primitive_roots'>1. Finding primitive roots of a prime</a>
<a href='#index'>Go back to the top</a>

+ If $ m $ is a primitive root modulo $ n $, then the multiplicative order of m is $ {\displaystyle \varphi (n)} $ i.e  $ a^{\phi (n)} \equiv 1 mod(n) $

+ We can use this to test a candidate m to see if it is primitive.

+ First, compute $ {\displaystyle \varphi (n)} $. Then determine the different prime factors of $ {\displaystyle \varphi (n)} $, say p1, ..., pk. 

+ Finally, compute $ {\displaystyle m^{\varphi (n)/p_{i}}{\bmod {n}}\qquad {\mbox{ for }}i=1,\ldots ,k}$ using a fast algorithm for modular exponentiation such as exponentiation by squaring. A number m for which these k results are all different from 1 is a primitive root.

In [29]:
def get_primitive_root(prime):
    # calculate phi(n)
    # phi(prime) is (prime-1)
    phi = prime-1
    prime_factors = list(pyecm.factors(phi, False, True, 10, 1))
    # set range for primitive root
    # so that we don't have to calculate all the roots
    d_range = np.random.randint(20,30)
    possible_root = 5
    while possible_root < prime:
        temp = []
        is_root = True
        for i in range(len(prime_factors)):
            remainder = gmpy2.powmod(possible_root,(phi//prime_factors[i]),prime)
            # if remainder is 1, then current number we're checking is not a primitive root
            if remainder == 1:
                is_root = False
                break
        if is_root and possible_root >= d_range:
            return possible_root
        possible_root=possible_root+1

# <a id='key'>2. Generation of Keys</a>
<a href='#index'>Go back to the top</a>

***Key Generation In El-Gamal :***

+ Choose a prime p using Crypto-Util
+ Choose e1 $ \in $ primitive_roots($ p $) 
    + Refer to <a href='#primitive_roots'>3. Finding primitive roots of a prime</a>
+ Choose $ d \in [2,p-2] $
+ Choose $ e2 = e1^d mod(p) $
    + Refer to <a href='#power_mod'>2. Modular exponentiation by squaring $ m^i mod(p) $</a>
+ Return ***Public keys :*** $ (e1,e2,p) $
+ Return ***Private key :*** $ (d) $

In [30]:
def generate_keys():
    # prime number of 25 digits i.e 84 bits
    n = 1000000000000000000000000
    random_add = np.random.randint(1,1000000)
    n = n + random_add
    p = int(gmpy2.next_prime(n))
    d = 0
    for i in range(2,p-1):
        select1 = np.random.randint(0,20)
        select2 = np.random.randint(0,20)
        if select1 == select2:
            d=i
            break
    e1 = get_primitive_root(p)
    e2 = gmpy2.powmod(e1,d,p)
    return e1,e2,p,d

# <a id='encrypt'>3. Encrypting plain_text blocks</a>
<a href='#index'>Go back to the top</a>

***Encyption in El-Gamal :***

***Input :*** $ Plain Text = [p_1,p_2,p_3,..........,p_k] $

***Output :*** $ Cipher Text = [c_1,c_2,c_3,..........,c_k] $

$ c_i = (c_{i1},c_{i2}) $

where

$ c_{i1} = e_1^{r} mod(p) $ and $ c_{i2} = ((e_2^{r})*p_i) mod(p) $ where $ i = 1,2.....,k $

***New $ r $ is used during every encryption to prevent any attacks***

In [31]:
def encrypt(plain_text_blocks,public_keys):
    cipher_text_blocks = []
    e1,e2,p = public_keys
    r = np.random.randint(2,50)
    print("\nr:",r)
    for plain_text in plain_text_blocks:
        cipher_text1 = gmpy2.powmod(e1,r,p)
        cipher_text2 = ((gmpy2.powmod(e2,r,p))*plain_text)%p
        cipher_text_blocks.append((cipher_text1,cipher_text2))
    return cipher_text_blocks

# <a id='decrypt'>4. Decrypting cipher_text</a>
<a href='#index'>Go back to the top</a>

***Decryption in El-Gamal :***

***Input :*** $ Cipher Text = [c_1,c_2,c_3,..........,c_k] $

***Output :*** $ Plain Text = [p_1,p_2,p_3,..........,p_k] $

$ p_{i} = ((c_{i1}^{-d})*c_{i2}) mod(p) $ where $ i = 1,2,...,k $

In [32]:
def decrypt(cipher_text_blocks,secret_key,public_keys):
    e1,e2,p = public_keys
    d = secret_key
    decypted_plain_text_blocks = []
    for cipher_text in cipher_text_blocks:
        cipher_text1,cipher_text2 = cipher_text
        cipher_text1 = gmpy2.powmod(cipher_text1,d,p)
        cipher_text1 = gmpy2.invert(cipher_text1,p)
        cipher_text2 = cipher_text2*cipher_text1
        plain_text = cipher_text2 % p
        decypted_plain_text_blocks.append(plain_text)
    return decypted_plain_text_blocks

# <a id='aux'>5. Auxiliary functions</a>
<a href='#index'>Go back to the top</a>

+ get_blocks() function converts the plain_text into blocks of plain_text by using the ***blocksize = 3***.
    + In this case, since prime used is 20 bits long i.e 7 digits long, it can encrypt upto 7 digit number
    + Since, each aplha-numeric character gives 2 digit plain_text, so we can use maximum of ***3 alphabets in a block***
+ format_plain_text() takes in the alphabetical plain_text blocks and returns their corresponding numeric plain_text blocks
+ def format_decrypted_plain_text() takes in the numeric_decrypted_plain_text blocks and return a single original plain text.

In [33]:
# taken in 'Hello World!!!' returns ['Hel','lo ','Wor','ld!','!!']
def get_blocks(PT,block_size):
    blocks = []
    i = 0
    while i<len(PT):
        temp_str=''
        if i+block_size-1 < len(PT):
            temp_str=temp_str+PT[i:i+block_size]
        else :
            temp_str=temp_str+PT[i::]
        blocks.append(temp_str)
        i=i+block_size
    return blocks
        
# covert plain_text block from characters to the numbers
def format_plain_text(PT):
    plain_text_blocks = []
    for block in PT:
        plain_text = 0
        for i in range(len(block)):
            # for 'd'
            if ord(block[i]) == 100:
                plain_text = plain_text*100 + 28
            # between (101,127)
            elif ord(block[i])>100:
                plain_text = plain_text*100 + (ord(block[i])-100)
            else :
                plain_text = plain_text*100 + (ord(block[i]))
        plain_text_blocks.append(plain_text)
    return plain_text_blocks

# convert numeric decypted_plain_text_blocks into a single plain text of characters
def format_decrypted_plain_text(decypted_plain_text_blocks):
    plain_text_blocks = []
    for dc_pt in decypted_plain_text_blocks:
        plain_text = ''
        temp = dc_pt
        # for 'd' temp = 28
        while temp > 0:
            if temp%100 == 28:
                plain_text = plain_text + 'd'
            elif (temp%100) in range(0,27):
                plain_text = plain_text + chr((temp%100)+100)
            else :
                plain_text = plain_text + chr((temp%100))
            temp = temp//100
        plain_text = plain_text[::-1] 
        plain_text_blocks.append(plain_text)
    final_plain_text = ''
    for plain_text_block in plain_text_blocks:
        final_plain_text = final_plain_text + plain_text_block
    return final_plain_text

# <a id='implementation'>6. Implementation</a>
<a href='#index'>Go back to the top</a>

### <a id='6.1'>6.1 Generating Keys</a>

In [34]:
e1,e2,p,d = generate_keys()

In [35]:
public_keys = (e1,e2,p)
secret_key = d
print("\nPublic Key :")
print('e1 :',e1)
print('e2 :',e2)
print('p :',p)
print("Secret Key :\nd :",d)


Public Key :
e1 : 33
e2 : 103441435086731682718531
p : 1000000000000000000854211
Secret Key :
d : 24


### <a id='6.2'>6.2 Generating plain_text</a>

In [36]:
PT = input("\nEnter Plain Text to encrypt : ")

original_plain_text = PT


Enter Plain Text to encrypt : Schrodinger's cat is dead and alive at the same time. This cat represents the (|psi_cat>=0.5*|psi_cat_alive> + 0.5*|psi_cat_dead>) i.e superposition of wave-functions representing cat being dead and alive at the same time.


In [37]:
block_size = 12
PT = get_blocks(PT,block_size)
print('\nPlain Text after converting to blocks',PT)


Plain Text after converting to blocks ["Schrodinger'", 's cat is dea', 'd and alive ', 'at the same ', 'time. This c', 'at represent', 's the (|psi_', 'cat>=0.5*|ps', 'i_cat_alive>', ' + 0.5*|psi_', 'cat_dead>) i', '.e superposi', 'tion of wave', '-functions r', 'epresenting ', 'cat being de', 'ad and alive', ' at the same', ' time.']


In [38]:
plain_text_blocks = format_plain_text(PT)
print('\nPlain text blocks after formatting to numbers:',plain_text_blocks)


Plain text blocks after formatting to numbers: [839904141128051003011439, 153299971632051532280197, 283297102832970805180132, 971632160401321597090132, 160509014632840405153299, 971632140112140115011016, 153216040132402412150595, 999716626148465342241215, 59599971695970805180162, 324332484653422412150595, 999716952801972862413205, 460132151712011412111505, 160511103211023219971801, 450217109916051110153214, 11214011501101605100332, 999716329801051003322801, 972832971028329708051801, 329716321604013215970901, 321605090146]


### <a id='6.3'>6.3 Encrypting plain_text to generate cipher_text</a>

In [39]:
cipher_text_blocks = encrypt(plain_text_blocks,public_keys)
print("\nCipher Text Blocks After El-Gamal encryption :",cipher_text_blocks)


r: 11

Cipher Text Blocks After El-Gamal encryption : [(mpz(50542106513726817), mpz(661963134578601353476006)), (mpz(50542106513726817), mpz(333166471840262813733456)), (mpz(50542106513726817), mpz(779080389800749467598945)), (mpz(50542106513726817), mpz(634040066868952112822104)), (mpz(50542106513726817), mpz(834379670376416829510185)), (mpz(50542106513726817), mpz(537528310516560150344972)), (mpz(50542106513726817), mpz(707020192838887142375559)), (mpz(50542106513726817), mpz(756174458299967416409923)), (mpz(50542106513726817), mpz(511953963743033305967736)), (mpz(50542106513726817), mpz(501127419720146001066490)), (mpz(50542106513726817), mpz(627562860235094724667896)), (mpz(50542106513726817), mpz(855773254002290700548161)), (mpz(50542106513726817), mpz(683369455057462161575308)), (mpz(50542106513726817), mpz(926770823009990023869435)), (mpz(50542106513726817), mpz(742378575165044177097731)), (mpz(50542106513726817), mpz(61360634019597105565900)), (mpz(50542106513726817), mpz(9360

### <a id='6.4'>6.4 Decrypting cipher_text to generate plain_text</a>

In [40]:
decypted_plain_text_blocks = decrypt(cipher_text_blocks,secret_key,public_keys)
print("\nPlain Text blocks after decryption of Cipher Text blocks :",decypted_plain_text_blocks)


Plain Text blocks after decryption of Cipher Text blocks : [mpz(839904141128051003011439), mpz(153299971632051532280197), mpz(283297102832970805180132), mpz(971632160401321597090132), mpz(160509014632840405153299), mpz(971632140112140115011016), mpz(153216040132402412150595), mpz(999716626148465342241215), mpz(59599971695970805180162), mpz(324332484653422412150595), mpz(999716952801972862413205), mpz(460132151712011412111505), mpz(160511103211023219971801), mpz(450217109916051110153214), mpz(11214011501101605100332), mpz(999716329801051003322801), mpz(972832971028329708051801), mpz(329716321604013215970901), mpz(321605090146)]


In [41]:
plain_text_after_decryption = format_decrypted_plain_text(decypted_plain_text_blocks)
print("\nAfter decryption Plain Text :",plain_text_after_decryption)


After decryption Plain Text : Schrodinger's cat is dead and alive at the same time. This cat represents the (|psi_cat>=0.5*|psi_cat_alive> + 0.5*|psi_cat_dead>) i.e superposition of wave-functions representing cat being dead and alive at the same time.


### <a id='6.5'>6.5 Checking if decrypted plain_text is same as original plain_text</a>

In [42]:
if (original_plain_text == plain_text_after_decryption):
    print("\nHurrayyy!!!\n\nDecrypted plain_text is same as original plain_text! :) ")


Hurrayyy!!!

Decrypted plain_text is same as original plain_text! :) 


# <a id='final'>7. Final Remarks</a>
<a href='#index'>Go back to the top</a>

+ ASCII values :
    + ASCII values from 0-31 are not alpha-numeric characters.
    + ASCII values from ***32-99 contain [A_Z,a,b,c] and special characters such as @,#,',",space,etc***
    + ASCII values from ***100-126 contain [d-z] which are 3 digit.***
    + Since we're ***not using ASCII values from 0-31, we can convert ASCII values of [d-z] to 2-digit number, by subtracting 100 from them.***
    + If we subtract 100 from ASCII values of 'd', which is 100, we get 00. ***dam will be 009709 which 9709***, so 'd' is lost in the processing. 
    + Unused ASCII values we have now are [27,28,29,30,31], so we assign d=28. 
+ This program uses 25 digit prime number, so it can encrypt upto maximum of 12 characters at a time i.e 24 digits
+ So, to encrypt large plain text, ***El-Gamal was implemented as a block cipher.***